In [1]:
import numpy as np
import pandas as pd
import scipy
import json
from collections import Counter
import copy
import random
import sklearn

In [2]:
%%time
# 입력데이터 로드
train = pd.read_json('train.json', typ = 'frame')
meta = pd.read_json('song_meta.json', typ = 'frame') # 707989
#sub_train = train.loc[:len(train)//10] # 11508
sub_train = train.loc[:len(train)//500]
#question = pd.read_json('arena_data/questions/val.json') # 2302
#question = question.loc[:len(question)//20]

CPU times: total: 10.1 s
Wall time: 10.2 s


In [15]:
# sub_train.to_json('sub_train.json', orient='records' , force_ascii=False)
# data file에 있는 train.json은 10분의 1 버전, 현재 폴더에 있는 train.json은 원본

In [3]:
# Own playlist test code

new_meta = meta[['id','artist_name_basket','song_gn_gnr_basket','song_name']]
new_meta["artist_name_basket"] = new_meta["artist_name_basket"].apply(lambda x: x[0] if x else x)

paulkim = new_meta[new_meta["artist_name_basket"] == "폴킴"]

ply1 = paulkim["id"].tolist()[:20]
ply2 = paulkim["id"].tolist()[20:40]
ply3 = paulkim["id"].tolist()[40:60]
playlists = [ply1,ply2,ply3]

C:\Users\User\AppData\Local\Temp\ipykernel_21332\3496926163.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_meta["artist_name_basket"] = new_meta["artist_name_basket"].apply(lambda x: x[0] if x else x)


In [6]:
# binary playlist by track matrix 만들기
num_playlist = len(sub_train)
num_songs = len(meta)
num_val = len(question)

playlist_by_track = np.zeros((num_playlist+3,num_songs), dtype=int)

for i, row in sub_train.iterrows():
    songs = row["songs"]
    for songid in songs:
        if songid < num_songs:
            playlist_by_track[i][songid] = 1

for i in range(3):
    songs = playlists[i]
    for songid in songs:
        playlist_by_track[num_playlist+i][songid] = 1


In [6]:
from scipy.sparse import csr_matrix, save_npz

# NumPy 배열을 CSR 희소 행렬로 변환
sparse_matrix = csr_matrix(playlist_by_track)

# 희소 행렬을 NPZ 파일로 저장
save_npz('playlist_by_track_sparse.npz', sparse_matrix)

In [2]:
from scipy.sparse import load_npz
# 저장된 sparse_matrix 파일 불러오기
loaded_sparse_matrix = load_npz('playlist_by_track_sparse.npz').toarray()
playlist_by_track = loaded_sparse_matrix

In [7]:
# val, question, answer는 URP/code/arena_data 폴더에 있음

from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=128)
tsvd = svd.fit_transform(playlist_by_track)

reconstructed_matrix = np.dot(tsvd, svd.components_)
#reconstructed_matrix = np.sort(reconstructed_matrix, axis=1)[:, ::-1]

In [8]:
# 기존에 1이없던 값 0으로 masking하기

one_index = [np.where(row > 0)[0] for row in playlist_by_track]

mask = np.zeros_like(reconstructed_matrix, dtype=bool)

for i, index in enumerate(one_index):
    mask[i, index] = True
    
reconstructed_matrix[mask] = 0

# TOP 100 SONGS 찾기
largest_indices = np.argsort(-reconstructed_matrix, axis=1)[:, :100]

In [16]:
answer1 = largest_indices[-3]
answer2 = largest_indices[-2]
answer3 = largest_indices[-1]
answers = [answer1,answer2,answer3]

array([656648,  93930, 523886, 561050,   6453, 200765, 433998, 168657,
       536290, 414836, 416950, 100818,  96170, 393260, 335324,  40271,
       693650, 117448, 423438, 267138, 370542, 559944, 180902,  14313,
       622947, 646865, 404290, 345115, 191473, 383208, 524734, 695248,
        52146, 323904, 261511, 114470,  53478, 521316, 235513, 453362,
       160386, 537220, 667211, 694099,  47587, 559688,  38790, 281630,
       522055, 348769, 122741, 562575, 265032, 693110, 153940, 456106,
        49666, 600807, 112069, 149555, 544355, 333443, 516249, 625154,
       336854, 158639, 405812, 655449, 593959, 434008, 243286, 338581,
       138571, 502732, 105370, 667587, 104182, 418935,  59636, 595717,
        28832, 505036, 109574, 517723, 506919, 459165, 505710, 264505,
       541682, 499085, 558822, 619251,   9091, 159289, 622756, 657469,
       566468, 248343, 669822, 478572], dtype=int64)

In [24]:
meta.head(1)

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0


In [52]:
for i in range(75,100,1):
    print(meta[meta["id"]==answer2[i]][["artist_name_basket","song_name"]].values.tolist())

[[['멜로우 노트'], '여름밤 너와 (Summernight , With You)']]
[[['Kazumi Tateishi Trio'], '歸らざる日日 / Kaerazaru Hibi (돌아올 수 없는 날들) (붉은 돼지 중에서)']]
[[['Yanni'], 'Reflections Of Passion']]
[[['제이레빗(J Rabbit)'], '인사 (Inst.)']]
[[['V.O.S'], '울어']]
[[['소정 (레이디스 코드)'], '우린 왜 이별 하는 걸까? (Better than Me)']]
[[['엠씨더맥스 (M.C the MAX)'], '잠시만 안녕 (Original)']]
[[['스웨덴세탁소'], 'foggy']]
[[['남우현'], '끄덕끄덕']]
[[['진원'], '고칠게']]
[[['스웨덴세탁소'], '안돼']]
[[['엠씨더맥스 (M.C the MAX)'], '입술의 말']]
[[['양다일', '효린'], '그리워']]
[[['먼데이 키즈 (Monday Kiz)', '김나영'], '눈물 (Tears)']]
[[['지코 (ZICO)', '소진 (Sojin)'], '아프다']]
[[['스웨덴세탁소'], '처음이라서 (With 이루마)']]
[[['바닐라 어쿠스틱'], '울컥해']]
[[['한경일'], '너는 간단하고 나는 복잡한 얘기']]
[[['KCM'], '5일']]
[[['에픽하이 (EPIK HIGH)'], 'Love Love Love (Feat. Yoong Jin Of Casker)']]
[[['프리스타일'], '수취인 불명']]
[[['스웨덴세탁소'], '기념일']]
[[['정키'], '잊혀지다 (Feat. 양다일)']]
[[['장덕철'], '그때, 우리로']]
[[['엠씨더맥스 (M.C the MAX)'], '1월']]


In [ ]:
for songid in answer2:
    meta[meta["id"]==songid][]

In [57]:
import psutil

available_memory = psutil.virtual_memory().available
print(f"Available Memory: {available_memory / (1024 ** 3):.2f} GiB")

Available Memory: 1.34 GiB
